### The Mandelbrot Set Iteration

The Mandelbrot set is given as the iteration of the following equations
$$
z_{n+1} = z^2_n + c
$$

Where:
* $z_0$ = 0 (starting value)
* $c$ is a complex number, where each unique $c$ will yield a different sequence of $z$ values

A point $c$ is in the Mandelbort set if, after iterating the equation multiple times, $|z|$ (the magnitude of $Z$) stays bounded (specifically, it remains $\leq$ 2). If $|z|$ escapes beyond 2, $c$ is not part of the set

In [8]:
import numpy as np
import matplotlib.pyplot as plt


def mandelbrot(c_points, max_iter, escape_radius) -> tuple[np.array, np.array]:
    '''
    This function calculates the number of iterations until the magnitude of z escapes to infinity. 
    Within each iteration the z is updated with c, an imaginary number representing a grid point. 
    Ultimately, a mandelbrot is calculated. 
    '''
    iteration_count = np.zeros(c_points.shape)
    mandelbrot_set = np.zeros(c_points.shape, dtype=bool)
    for i in range(c_points.shape[0]):
        for j in range(c_points.shape[1]):

            # take a gridpoint
            c = c_points[i, j]
            z = 0
            for iteration in range(max_iter):

                # update of z
                z = z**2 + c
                if abs(z) > escape_radius:
                    mandelbrot_set[i, j] = False
                    iteration_count[i, j] =iteration
                    break
            else:
                mandelbrot_set[i, j] = True
                iteration_count[i, j] = max_iter
    return (mandelbrot_set, iteration_count)


            

In [ ]:
# grid_size = np.linspace(4, )
import os
from multiprocessing import Pool
from worker import worker_function

def partition_grid(grid, proc):
    grid_size = grid//proc
    rest = grid % proc

    indexes =[]
    index = 0

    real = np.linspace(-2.0, 1.0, grid)
    imag = np.linspace(-1.5, 1.5, grid)

    # Create a 2D grid of complex numbers c
    real_grid, imag_grid = np.meshgrid(real, imag)
    c_points = real_grid + 1j * imag_grid

    for _ in range(proc):
        if rest > 0:
            addit = 1
            rest -=1
        else: 
            addit = 0
        indexes.append((index, index+grid_size+addit))
        index += grid_size + addit
    
    c_slices = [c_points[start:end] for start, end in indexes]
    return c_slices


def driver_func(grid, s):

    # max 10 processes
    PROCESSES = 10 
    with Pool(PROCESSES) as pool:  # Adjust the number of processes as needed 
        pid = os.getpid()
        # print(f"Current Process ID: {pid}")

        # every process gets a fraction of the grid (row-wise partitioning)
        c_parts = partition_grid(grid, PROCESSES)

        args = [(c_slice, s) for c_slice in c_parts]
        results = pool.map(worker_function,  args)
        total_points_parts, inside_points_parts = zip(*results)
        total_points = sum(total_points_parts)
        inside_points= sum(inside_points_parts)
        print(f"Result: {results}\nTotal points and points inside mandelbrot: {total_points, inside_points}")
    return total_points, inside_points

if __name__ == '__main__':
    s_values = [10, 20, 50, 100, 150, 250, 500, 1000, 2000]
    grids = [10, 50, 100, 250, 500, 1000, 2000, 5000]
    
    saved_values = {}
    # make sure the number of procs you use does not exceed the processor count
    print(f"Number of available CPU cores: {os.cpu_count()}")
    for grid in grids:
        for s_value in s_values:
            key = (grid, s_value)
            tot_points, in_points = driver_func(grid, s_value)
            saved_values[key] = (tot_points, in_points)

#Since the mandelbrot is symmetrical, we could halve the grid and get the same fraction 

In [6]:
with open("question2.txt", "w") as file:
    file.write("grid_size, max_iterations, total_points, points_inside\n")
    for (key1, key2), (value1, value2)  in saved_values.items():
        file.write(f"{key1}, {key2}, {value1}, {value2}\n")


In [ ]:
import pandas as pd
df = pd.read_csv("question2.txt", delimiter=",", skiprows=1,
                 names=["grid_size", "max_iterations", "total_points", "points_inside"])
df["fraction_mand"] = 1 - df["points_inside"]/df["total_points"]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
for max_iter, group in df.groupby("max_iterations"):
    ax1.plot(group["grid_size"], group["fraction_mand"], label=f"{max_iter}", marker= 'o')

# ax1.set_xlabel("Grid Size")
ax1.set_ylabel("Fraction of Points Inside Mandelbrot")
ax1.set_title("Full Size")
ax1.set_xlabel("Grid Size")
ax1.legend(title="Iteration Bound")
ax1.set_xlim(0, 5040)
ax1.grid(True)
# ax1.show()

# plt.figure(figsize=(10, 6))
for max_iter, group in df.groupby("max_iterations"):
    ax2.plot(group["grid_size"], group["fraction_mand"], label=f"{max_iter}", marker= 'o')

ax2.set_xlabel("Grid Size")
# ax2.set_ylabel("Fraction of Points Inside Mandelbrot")
ax2.set_title("Zoomed In")
# ax2.legend(title="bound on number of iterations")
ax2.set_xlim(0, 5040)
ax2.grid(True)
ax2.set_ylim(0.165, 0.175)
# ax2.show()

# plt.figure(figsize=(10, 6))
for max_iter, group in df.groupby("max_iterations"):
    ax3.plot(group["grid_size"], group["fraction_mand"], label=f"{max_iter}", marker= 'o')

ax3.set_xlabel("Grid Size")
# ax3.set_ylabel("Fraction of Points Inside Mandelbrot")
ax3.set_title("Zoomed In")
# 3x2.legend(title="bound on number of iterations")
ax3.set_xlim(0, 5040)
ax3.grid(True)
ax3.set_xlim(0, 500)
# ax2.show()


fig.suptitle("Fraction of Points Inside Mandelbrot vs Grid-Size")
plt.tight_layout()
plt.show()

#### Making a Mandelbrot figure

In [ ]:
# Setting up the imaginary point set
real = np.linspace(-2.0, 1.0, 1000)
imag = np.linspace(-1.5, 1.5, 1000)

# Create a 2D grid of complex numbers c
real_grid, imag_grid = np.meshgrid(real, imag)
c_points = real_grid + 1j * imag_grid

max_iter = 250
escape_radius = 2

mandel_set, iter_count = mandelbrot(c_points, max_iter, escape_radius)





plt.figure(figsize=(12, 8), dpi=300)
plt.imshow(iter_count, extent=(-2.0, 1.0, -1.5, 1.5), cmap='plasma', origin='lower')
plt.xlabel("Re(c)")
plt.ylabel("Im(c)")
plt.title("Mandelbrot Set")
plt.colorbar(label="In Mandelbrot Set")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import colorsys

def mandelbrot_iteration(c, max_iter, escape_radius):
    z = 0
    for n in range(max_iter):
        z = z*z + c
        if abs(z) > escape_radius:
            return n
    return max_iter

def compute_mandelbrot(real, imag, max_iter, escape_radius):
    height, width = len(imag), len(real)
    iteration_count = np.zeros((height, width), dtype=np.int64)
    
    # Create a mesh grid for vectorized computation
    real_grid, imag_grid = np.meshgrid(real, imag)
    c = real_grid + 1j * imag_grid
    z = np.zeros_like(c, dtype=np.complex128)
    
    for i in range(max_iter):
        mask = np.abs(z) <= escape_radius
        z[mask] = z[mask]**2 + c[mask]
        iteration_count[mask] = i
    
    # Set points that never escaped to max_iter
    iteration_count[np.abs(z) <= escape_radius] = max_iter
    
    return iteration_count

def create_custom_colormap():
    # Create colors for our custom colormap
    colors = []
    for i in range(256):
        # Convert HSV to RGB, cycling through hues
        hue = i/256
        saturation = 1.0
        value = 1.0 if i > 10 else i/10.0  # Darker colors for low iteration counts
        rgb = colorsys.hsv_to_rgb(hue, saturation, value)
        colors.append(rgb)
    
    # Add black for the Mandelbrot set itself
    colors.append((0, 0, 0))
    
    return LinearSegmentedColormap.from_list('custom', colors)

def plot_mandelbrot(iteration_count, real_range, imag_range, max_iter, 
                   zoom_center=None, zoom_width=None, title="Enhanced Mandelbrot Set"):
    plt.figure(figsize=(15, 10), dpi=300)
    
    # Create the plot with a custom colormap
    custom_cmap = create_custom_colormap()
    
    # Normalize iteration counts for smoother color transitions
    smooth_iter = iteration_count + 1 - np.log(np.log(np.abs(iteration_count)))/np.log(2)
    smooth_iter[iteration_count == max_iter] = max_iter
    
    # Plot with enhanced aesthetics
    plt.imshow(smooth_iter, 
              extent=(real_range[0], real_range[-1], imag_range[0], imag_range[-1]),
              cmap=custom_cmap,
              origin='lower',
              aspect='equal')
    
    # Add gridlines
    plt.grid(True, alpha=0.3, linestyle='--')
    
    # Enhance labels and title
    plt.xlabel("Re(c)", fontsize=12)
    plt.ylabel("Im(c)", fontsize=12)
    plt.title(title, fontsize=14, pad=20)
    
    # Add colorbar with custom label
    cbar = plt.colorbar(label="Iteration Count", pad=0.02)
    cbar.ax.set_ylabel("Iteration Count", fontsize=10)
    
    # Add zoom box if specified
    if zoom_center and zoom_width:
        zoom_rect = plt.Rectangle(
            (zoom_center[0] - zoom_width/2, zoom_center[1] - zoom_width/2),
            zoom_width, zoom_width,
            fill=False, color='white', linestyle='--'
        )
        plt.gca().add_patch(zoom_rect)
    
    plt.tight_layout()
    return plt.gcf()

# Parameters for the main plot
# Reduced resolution for faster computation since we don't have Numba
real = np.linspace(-2.0, 1.0, 1000)  
imag = np.linspace(-1.5, 1.5, 1000)
max_iter = 1000
escape_radius = 2.0

# Compute the main Mandelbrot set
iteration_count = compute_mandelbrot(real, imag, max_iter, escape_radius)

# Create main plot
main_plot = plot_mandelbrot(iteration_count, real, imag, max_iter, 
                           title="The Mandelbrot Set")
plt.show()

# Create a zoomed plot of an interesting region
zoom_center = (-0.7435, 0.1314)
zoom_width = 0.002

# Calculate new ranges for zoom
zoom_real = np.linspace(zoom_center[0] - zoom_width/2, 
                       zoom_center[0] + zoom_width/2, 1000)
zoom_imag = np.linspace(zoom_center[1] - zoom_width/2, 
                       zoom_center[1] + zoom_width/2, 1000)

# Compute zoomed region
zoom_iteration_count = compute_mandelbrot(zoom_real, zoom_imag, max_iter, escape_radius)

# Create zoomed plot
zoom_plot = plot_mandelbrot(zoom_iteration_count, zoom_real, zoom_imag, max_iter,
                           title=f"Mandelbrot Set Zoom (width={zoom_width:.6f})")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.qmc import LatinHypercube

# Define s_values and grids (from exercise 2)
s_values = [10, 20, 50, 100, 150, 250, 500, 1000, 2000]
grids = [10, 50, 100, 250, 500, 1000, 2000, 5000]
#s_values = [20, 200, 500]
#grids = [10, 50, 100]

def mandelbrot(c_points, max_iter, escape_radius) -> tuple[int, int]:
    '''
    This function calculates the number of iterations until the magnitude of z escapes to infinity. 
    Within each iteration the z is updated with c, an imaginary number representing a grid point. 
    Ultimately, a mandelbrot is calculated. 
    '''
    # iteration_count = np.zeros(c_points.shape)
    # mandelbrot_set = np.zeros(c_points.shape, dtype=bool)
    number_inside = 0
    total_numbers = 0
    for i in range(c_points.shape[0]):
        for j in range(c_points.shape[1]):
            total_numbers +=1
            # take a gridpoint
            c = c_points[i, j]
            z = 0
            for iteration in range(max_iter):
                # update of z
                z = z**2 + c
                if abs(z) > escape_radius:
                    number_inside+=1
                    # mandelbrot_set[i, j] = False
                    # iteration_count[i, j] =iteration
                    break
            # else:
                # mandelbrot_set[i, j] = True
                # iteration_count[i, j] = max_iter
    return (total_numbers, number_inside)

def latin_hypercube_sampling(s_values, max_iter=100, escape_radius=2, scramble=True, seed=None):

    # Initialize the LatinHypercube sampler for a 2D grid
    sampler = LatinHypercube(d=2, scramble=scramble, seed=seed)
    sample = sampler.random(n=s_values)

    real_range = np.linspace(-2.0, 1.0, grid)
    imag_range = np.linspace(-1.5, 1.5, grid)
    
    real_samples = real_range[0] + sample[:, 0] * (real_range[-1] - real_range[0])
    imag_samples = imag_range[0] + sample[:, 1] * (imag_range[-1] - imag_range[0])

    complex_samples_LHS = real_samples + 1j * imag_samples 

    return complex_samples_LHS

for s in s_values:
    for grid in grids:

        complex_samples_LHS = latin_hypercube_sampling(s_values=s, max_iter=100, escape_radius=2).reshape((s, 1))
        total_points, number_inside = mandelbrot(complex_samples_LHS, max_iter=100, escape_radius=2)

        #print(f"s_values: {s}, Grid size: {grid}x{grid}, Total points: {total_points}, Points inside: {number_inside}")
